## Problem Set 1 - BDMLAE

In [1]:
#Packages:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import scipy as sc
from scipy.stats import bootstrap
from sklearn.impute import KNNImputer

In [2]:
#Setting work space and master data frame:
bd=pyr.read_r("C:/Users/ASUS/Documents/Universidad de los Andes/2022-2/Big Data and Machine Learning for Applied Economics/Taller 1/PS1_BD-ML/bdPS1.Rdata")
print(bd.keys()) #Load database.
df0=bd["df"]
df=df0.drop(df0[df0.age<18].index) #Restrict the data to legal age individuals (age>=18).
df=df.drop(df[df.ocu==0].index) #Restrict the data to employed individuals (ocu==1).
dfsi=(df[["age", "maxEducLevel", "sizeFirm", "oficio", "hoursWorkUsual", "formal", "ingtot", "mes", "estrato1", "sex", 
"regSalud", "p6426", "p7090","p7510s7a1", "impa", "isa", "impaes", "isaes", "fex_c", "fweight", "relab"]]) #Relevant variables selection. 

odict_keys(['df'])


In [3]:
dfsi["impa"].replace(0, 1, inplace=True)
dfsi["impa"].replace(np.nan, 0, inplace=True)
dfsi["impaes"].replace(0, 1, inplace=True)
dfsi["impaes"].replace(np.nan, 0, inplace=True)
dfsi["isa"].replace(0, 1, inplace=True)
dfsi["isa"].replace(np.nan, 0, inplace=True)
dfsi["isaes"].replace(0, 1, inplace=True)
dfsi["isaes"].replace(np.nan, 0, inplace=True)

dfsi["impacomp"]=dfsi["impa"]+dfsi["impaes"]
dfsi["isacomp"]=dfsi["isa"]+dfsi["isaes"]

dfsi['impacomp']=dfsi['impacomp'].replace(0,np.nan)
dfsi['isacomp']=dfsi['isacomp'].replace(0,np.nan)
dfsi['impacomp']=dfsi['impacomp'].replace(1, 0)
dfsi['isacomp']=dfsi['isacomp'].replace(1, 0)
dfsi['impacomp']=dfsi['impacomp'].replace(2, 0)
dfsi['isacomp']=dfsi['isacomp'].replace(2, 0)

dfsi.isnull().sum() 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10288\1487940839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsi["impa"].replace(0, 1, inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10288\1487940839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsi["impa"].replace(np.nan, 0, inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10288\1487940839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsi["impaes"].r

age                  0
maxEducLevel         1
sizeFirm             0
oficio               0
hoursWorkUsual       0
formal               0
ingtot               0
mes                  0
estrato1             0
sex                  0
regSalud          1420
p6426                0
p7090                0
p7510s7a1            0
impa                 0
isa                  0
impaes               0
isaes                0
fex_c                0
fweight              0
relab                0
impacomp           248
isacomp             31
dtype: int64

In [4]:
#Missing values:
imputer=KNNImputer(n_neighbors=19) 
imputer.fit(dfsi)
dfi=pd.DataFrame(imputer.transform(dfsi))
dict=({0: "age", 1: "maxEducLevel", 2: "sizeFirm", 3: "oficio", 4: "hoursWorkUsual", 5: "formal", 6: "ingtot", 7: "mes", 8: "estrato1", 
9: "sex", 10: "regSalud", 11: "p6426", 12: "p7090", 13: "p7510s7a1", 14: "impa", 15: "isa", 16: "impaes", 17: "isaes", 18: "fex_c", 
19: "fweight", 20: "relab", 21: "impacomp", 22: "isacomp"})
dfi.rename(columns=dict, inplace=True)

In [5]:
#Convert categorical variables to dummy variables:
oficio_d=pd.get_dummies(dfi["oficio"], prefix="oficio") 
sizeFirm_d=pd.get_dummies(dfi["sizeFirm"], prefix="size") 
estrato1_d=pd.get_dummies(dfi["estrato1"], prefix="estrato") 
regSalud_d=pd.get_dummies(dfi["regSalud"], prefix="regs") 
maxeduc_d=pd.get_dummies(dfi["maxEducLevel"], prefix="educ") 
relab_d=pd.get_dummies(dfi["relab"], prefix="relab") 

#Merge dummy's variables data frame with master data frame:
dfi=pd.merge(dfi, oficio_d, left_index=True, right_index=True) 
dfi=pd.merge(dfi, sizeFirm_d, left_index=True, right_index=True)
dfi=pd.merge(dfi, estrato1_d, left_index=True, right_index=True)
dfi=pd.merge(dfi, regSalud_d, left_index=True, right_index=True)
dfi=pd.merge(dfi, maxeduc_d, left_index=True, right_index=True)
dfi=pd.merge(dfi, relab_d, left_index=True, right_index=True)
dfi["p7090"]=np.where(dfi["p7090"]==2, 0, 1)

In [6]:
#Transform categorial atribute variables to numerical variable: 
dfi["hoursWorkUsual"]=pd.to_numeric(dfi["hoursWorkUsual"])
dfi["formal"]=pd.to_numeric(dfi["formal"])
dfi["p6426"]=pd.to_numeric(dfi["p6426"])

#Descriptive statistics data frame:
ds=(dfi[["age", "hoursWorkUsual", "p6426", "sex", "formal", "p7090", "estrato_1.0", "estrato_2.0", "estrato_3.0", "estrato_4.0", 
"estrato_5.0", "estrato_6.0", "regs_1.0", "regs_2.0", "regs_3.0", "educ_1.0", "educ_3.0", "educ_4.0", "educ_5.0", 
"educ_6.0", "educ_7.0"]].describe(include="all"))
ds=ds.T
ds=ds[["count", "mean", "std", "min", "50%", "max"]]
print(ds.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &    count &       mean &        std &   min &   50\% &    max \\
\midrule
age            &  16542.0 &  39.436162 &  13.482844 &  18.0 &  38.0 &   94.0 \\
hoursWorkUsual &  16542.0 &  47.007738 &  15.542736 &   1.0 &  48.0 &  130.0 \\
p6426          &  16542.0 &  63.757949 &  89.487752 &   0.0 &  24.0 &  720.0 \\
sex            &  16542.0 &   0.529984 &   0.499115 &   0.0 &   1.0 &    1.0 \\
formal         &  16542.0 &   0.586809 &   0.492421 &   0.0 &   1.0 &    1.0 \\
p7090          &  16542.0 &   0.096844 &   0.295755 &   0.0 &   0.0 &    1.0 \\
estrato\_1.0    &  16542.0 &   0.107182 &   0.309353 &   0.0 &   0.0 &    1.0 \\
estrato\_2.0    &  16542.0 &   0.418087 &   0.493260 &   0.0 &   0.0 &    1.0 \\
estrato\_3.0    &  16542.0 &   0.361625 &   0.480485 &   0.0 &   0.0 &    1.0 \\
estrato\_4.0    &  16542.0 &   0.068069 &   0.251872 &   0.0 &   0.0 &    1.0 \\
estrato\_5.0    &  16542.0 &   0.019345 &   0.137738 &   0.0 &   0.0 &    1.0 \\
est

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10288\3537429800.py:12: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ds.to_latex())
